# Molecular Dynamics
## Initializing a system to a specific density using HOOMD 3

HOOMD 3 is still in beta. This tutorial requires compiling hoomd from [source](https://github.com/glotzerlab/hoomd-blue) on the branch `/feature/new-object-API`. More instructions can be found in [the hoomd docs](https://hoomd-blue.readthedocs.io/en/feature-new-object-api/installation.html).

mbuild was installed from [PR 698](https://github.com/mosdef-hub/mbuild/pull/698). 

and the environment was created using:
```
conda create -n cmelab-hoomd3 -c conda-forge -c mosdef -c omnia 'python=3.7' pip matplotlib numpy scipy packmol 'nglview>=2.7' oset parmed mdtraj foyer openbabel py3Dmol 'nodejs>=10' jupyterlab unyt
```

<a id="toc"></a>
## Table of Contents
- [initialize a system](#init)

In [1]:
import foyer
import hoomd
import matplotlib.pyplot as plt
import mbuild as mb
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import numpy as np
import parmed as pmd

<a id="init"></a>
## Starting Structure


[back to top](#toc)

In [2]:
propane = mb.load("CCC", smiles=True)

propane.visualize().show()

/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
/Users/jenny/Projects/mbuild/mbuild/compound.py:2749: UserWarning: No unitcell detected for pybel.Molecule CCC	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In the above visualization, carbon atoms are colored grey and hydrogens are white. 

Let's say we want a system of liquid propane at a density of 0.6 g/mL.
mBuild and foyer use these [units](http://docs.openmm.org/7.2.0/userguide/theory.html#units) (nm, amu)

1 amu = 1.66054e-24 g

1 mL = 1 cm$^{3}$ = 1e21 nm$^{3}$

For simplicity, let's say that each propane molecule is 44 amu (3 carbons @12 amu + and 8 hydrogens @1 amu)

If our box volume is 2x2x2 nm (8 nm$^{3}$), then how many propane molecules do we need to fill it at this density?

In [3]:
target_density = 0.6 /1e21 / 1.66054e-24
print(f"The target density is {target_density:.2f} in amu/nm^3")

number = int(round(target_density * 8 / 44))
print(f"This means we need {number} propane molecules to fill a 2x2x2 nm box")

The target density is 361.33 in amu/nm^3
This means we need 66 propane molecules to fill a 2x2x2 nm box


/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
box = mb.Box([2,2,2])
system_box = mb.packing.fill_box(propane, number, box=box)
system_box.visualize().show()

/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<a id="ff"></a>
## Apply forcefield


_Side note: The `foyer.Forcefield.apply` function currently generates a [parmed structure](https://parmed.github.io/ParmEd/html/structure.html), which does not have the same convenient system-building operations as mbuild. This is why we can't fill the box with our typed molecule. The typed molecule was shown above simply to demonstrate which atoms are assigned the different types. With the new [GMSO](https://gmso.mosdef.org/en/latest/) back-end, this workflow may change._

[back to top](#toc)

In [5]:
oplsaa = foyer.forcefields.load_OPLSAA()
box_struc = oplsaa.apply(system_box, assert_dihedral_params=False)

/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/foyer/forcefield.py:449: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/foyer/forcefield.py:461: UserWarning: No force field name found in force field XML file.
  'No force field name found in force field XML file.'
/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You hav

<a id="run"></a>
## Run simulation

Now that we have our simulation volume, we can set up our MD simulation to run in [HOOMD](https://hoomd-blue.readthedocs.io/en/stable/) using mbuild's `create_hoomd_simulation` funciton. This will read in the atom positions, bonding information, and forcefield parameters that we have already set. Additionally we need to tell HOOMD about the temperature ([kT, really the thermal energy](https://hoomd-blue.readthedocs.io/en/stable/units.html)), the thermostat coupling (tau), the cutoff where the particles no longer "feel" each other (r_cut), and the time step (dt). Everything else set in the cell below is telling HOOMD how often/when to write out data. The [gsd](https://gsd.readthedocs.io/en/stable/) files will contain the simulation snapshots and the log file will record quantities as the simulation progresses. 

[back to top](#toc)

In [6]:
snapshot, forcefield, nlist, values = create_hoomd_simulation(
    box_struc, r_cut=1.2, auto_scale=True
)

Processing LJ and QQ
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing RB torsions
HOOMD SimulationContext updated from ParmEd Structure


/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
langevin = hoomd.md.methods.Langevin(
    filter=hoomd.filter.All(), kT=1.0, seed=1
)

integrator = hoomd.md.Integrator(
    dt=0.0001, methods=[langevin], forces=forcefield
)

gsd = hoomd.dump.GSD(
    filename='traj.gsd', 
    trigger=hoomd.trigger.Periodic(int(1e5)),
    overwrite=True
)

/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
sim = hoomd.Simulation(hoomd.device.CPU())
sim.create_state_from_snapshot(snapshot)
sim.operations.integrator = integrator
sim.operations.add(gsd)
sim.operations.schedule()

sim.run(3e5)

/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
*Warning*: pos 16:-2.31951 -2.86367457143 -0.711845714286
*Warning*: fractional pos :0.09408575 -0.00114305 0.375427
*Warning*: lo: -2.85714285714 -2.85714285714 -2.85714285714
*Warning*: hi: 2.85714285714 2.85714285714 2.85714285714
*Warning*: Not all particles were found inside the given box


RuntimeError: Error initializing ParticleData

In [4]:
box = mb.Box([5,5,5])
system_box = mb.packing.fill_box(propane, number, box=box)
system_box.visualize().show()

/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
oplsaa = foyer.forcefields.load_OPLSAA()
box_struc = oplsaa.apply(system_box, assert_dihedral_params=False)

/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/foyer/forcefield.py:449: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/foyer/forcefield.py:461: UserWarning: No force field name found in force field XML file.
  'No force field name found in force field XML file.'
/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You hav

In [6]:
snapshot, forcefield, nlist, values = create_hoomd_simulation(
    box_struc, r_cut=1.2, auto_scale=True
)

Processing LJ and QQ
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing RB torsions
HOOMD SimulationContext updated from ParmEd Structure


/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
langevin = hoomd.md.methods.Langevin(
    filter=hoomd.filter.All(), kT=1.0, seed=1
)

integrator = hoomd.md.Integrator(
    dt=0.0001, methods=[langevin], forces=forcefield
)

gsd = hoomd.dump.GSD(
    filename='traj.gsd', 
    trigger=hoomd.trigger.Periodic(int(1e5)),
    overwrite=True
)

/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
box1 = hoomd.Box(5,5,5)
box2 = hoomd.Box(2,2,2)
#ramp = hoomd.variant.Ramp(2, 5, 100, 500)
#hoomd.update.BoxResize(box1, box2, ramp, trigger, scale_particles=True)
compress = hoomd.update.BoxResize.linear_volume(
    box1, box2, 100, 1000, hoomd.trigger.Periodic(int(50)), scale_particles=False
)

/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
sim = hoomd.Simulation(hoomd.device.CPU())
sim.create_state_from_snapshot(snapshot)
sim.operations.integrator = integrator
sim.operations.add(gsd)
sim.operations.updaters.append(compress)
sim.operations.schedule()

sim.run(3e5)

/Users/jenny/miniconda3/envs/cmelab-hoomd3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'sim' is not defined